# Example Seldon Core Deployments using Ksonnet


## Prerequistes

You will need

 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [ksonnet client](https://ksonnet.io/)

## Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

** If running on GCP then run following to add cluster-admin to your user account **

In [1]:
!kubectl create clusterrolebinding my-cluster-admin-binding --clusterrole=cluster-admin --user=$(gcloud info --format="value(config.account)")

clusterrolebinding.rbac.authorization.k8s.io/my-cluster-admin-binding created


On most clusters you will probably need to add cluster-admin privledges

In [2]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [3]:
!kubectl create namespace seldon

namespace/seldon created


In [4]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry. If you run into a ```403 API rate limit``` error, follow the instructions on [this page](https://github.com/ksonnet/ksonnet/blob/master/docs/troubleshooting.md) to resolve it.

In [5]:
!rm -rf my-ml-deployment && ks init my-ml-deployment 

INFO Using context "minikube" from kubeconfig file "/home/clive/.kube/config" 
INFO Creating environment "default" with namespace "seldon", pointing to "version:v1.13.3" cluster at address "https://192.168.39.236:8443" 
INFO Generating ksonnet-lib data at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/lib/ksonnet-lib/v1.13.3' 


In [6]:
!cd my-ml-deployment && \
    ks registry add seldon-core ../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --withApife=false --withAmbassador=true --singleNamespace=true --namespace=seldon --withRbac=true

INFO Retrieved 19 files                           
INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/seldon-core.jsonnet' 


In [7]:
!cd my-ml-deployment && \
      ks apply default

INFO Applying serviceaccounts seldon.seldon       
INFO Creating non-existent serviceaccounts seldon.seldon 
INFO Applying roles seldon.seldon-local           
INFO Creating non-existent roles seldon.seldon-local 
INFO Applying rolebindings seldon.seldon          
INFO Creating non-existent rolebindings seldon.seldon 
INFO Applying services seldon.seldon-core-ambassador 
INFO Creating non-existent services seldon.seldon-core-ambassador 
INFO Applying services seldon.seldon-core-redis   
INFO Creating non-existent services seldon.seldon-core-redis 
INFO Applying customresourcedefinitions seldondeployments.machinelearning.seldon.io 
INFO Creating non-existent customresourcedefinitions seldondeployments.machinelearning.seldon.io 
INFO Applying roles seldon.ambassador             
INFO Creating non-existent roles seldon.ambassador 
INFO Applying rolebindings seldon.ambassador      
INFO Creating non-existent rolebindings seldon.ambassador 
INFO Applying deployments seldon.seldon-core-seldo

In [8]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-ambassador

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Serve Single Model

In [9]:
!cd my-ml-deployment && \
    ks generate seldon-serve-simple-v1alpha2 mymodel --image seldonio/mock_classifier:1.0 && \
    ks apply default -c mymodel

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/mymodel.jsonnet' 
INFO Applying seldondeployments seldon.mymodel    
INFO Creating non-existent seldondeployments seldon.mymodel 


In [10]:
!cd my-ml-deployment && \
    ks show default -c mymodel

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: mymodel
  name: mymodel
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: mymodel
  name: mymodel
  oauth_key: ""
  oauth_secret: ""
  predictors:
  - annotations:
      predictor_version: v1
    componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: mymodel
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    graph:
      children: []
      endpoint:
        type: REST
      name: mymodel
      type: MODEL
    name: mymodel
    replicas: 1


In [11]:
!kubectl rollout status deploy/mymodel-mymodel-025d03d

Waiting for deployment "mymodel-mymodel-025d03d" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-025d03d" successfully rolled out


### Get predictions

In [15]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="seldon")

#### REST Request

In [18]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.3077357174056141
  }
}

Response:
meta {
  puid: "r2g9qc0a9v3ovvbq20mmqhui4k"
  requestPath {
    key: "mymodel"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.0685658467096764
  }
}



#### gRPC Request

In [19]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.3320692161726353
  }
}

Response:
meta {
  puid: "eqrnsmp4jpvjnestpnlba0a2ba"
  requestPath {
    key: "mymodel"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07013630483204733
  }
}



In [20]:
!cd my-ml-deployment && \
    ks delete default -c mymodel && \
    ks component rm mymodel

INFO Deleting seldondeployments seldon.mymodel    
INFO removing environment component                component-name=mymodel
INFO Removing component parameter references ...  
INFO Deleting component "mymodel"                 
INFO Successfully deleted component 'mymodel'     


## Serve AB Test

In [21]:
!cd my-ml-deployment && \
    ks generate seldon-abtest-v1alpha2 myabtest --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 && \
    ks apply default -c myabtest

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/myabtest.jsonnet' 
INFO Applying seldondeployments seldon.myabtest   
INFO Creating non-existent seldondeployments seldon.myabtest 


In [22]:
!cd my-ml-deployment && \
    ks show default -c myabtest

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: myabtest
  name: myabtest
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: myabtest
  name: myabtest
  oauth_key: ""
  oauth_secret: ""
  predictors:
  - componentSpecs:
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-1
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-2
          volumeMounts: []
        terminationGracePeriodSeconds: 1
        volumes: []
    graph:
      children:
      - children: []
  

In [23]:
!kubectl rollout status deploy/myabtest-myabtest-41de5b8
!kubectl rollout status deploy/myabtest-myabtest-df66c5c

deployment "myabtest-myabtest-41de5b8" successfully rolled out
deployment "myabtest-myabtest-df66c5c" successfully rolled out


### Get predictions

In [25]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="myabtest",namespace="seldon")

#### REST Request

In [26]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.34230611444081804
  }
}

Response:
meta {
  puid: "v24do7e1ehu7j4v2ancsa7vkl0"
  routing {
    key: "random-ab-test"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "random-ab-test"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07080687166429585
  }
}



#### gRPC Request

In [27]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.8980743238334664
  }
}

Response:
meta {
  puid: "hpprtlr3vnmrm7p0sajgpsfqsh"
  routing {
    key: "random-ab-test"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "random-ab-test"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.11726462409634174
  }
}



In [28]:
!cd my-ml-deployment && \
    ks delete default -c myabtest && \
    ks component rm myabtest

INFO Deleting seldondeployments seldon.myabtest   
INFO removing environment component                component-name=myabtest
INFO Removing component parameter references ...  
INFO Deleting component "myabtest"                
INFO Successfully deleted component 'myabtest'    


## Serve Multi-Armed Bandit

In [29]:
!cd my-ml-deployment && \
    ks generate seldon-mab-v1alpha2 mymab --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 && \
    ks apply default -c mymab

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/mymab.jsonnet' 
INFO Applying seldondeployments seldon.mymab      
INFO Creating non-existent seldondeployments seldon.mymab 


In [30]:
!cd my-ml-deployment && \
    ks show default -c mymab

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: mymab
  name: mymab
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: mymab
  name: mymab
  oauth_key: ""
  oauth_secret: ""
  predictors:
  - componentSpecs:
    - metadata:
        labels:
          version: v1
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-1
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-2
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - spec:
        containers:
 

In [31]:
!kubectl rollout status deploy/mymab-mymab-41de5b8
!kubectl rollout status deploy/mymab-mymab-b8038b2
!kubectl rollout status deploy/mymab-mymab-df66c5c

Waiting for deployment "mymab-mymab-41de5b8" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymab-mymab-41de5b8" successfully rolled out
deployment "mymab-mymab-b8038b2" successfully rolled out
deployment "mymab-mymab-df66c5c" successfully rolled out


### Get predictions

In [32]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymab",namespace="seldon")

#### REST Request

In [33]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.9709451633810755
  }
}

Response:
meta {
  puid: "7iupu7jv64h11s3fn28nt0did0"
  routing {
    key: "eg-router"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.12502063459073778
  }
}



#### gRPC Request

In [36]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.5866844650685953
  }
}

Response:
meta {
  puid: "5j836klt3kvsiqj6ujm7rho59d"
  routing {
    key: "eg-router"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08867010149238716
  }
}



In [37]:
!cd my-ml-deployment && \
    ks delete default -c mymab && \
    ks component rm mymab

INFO Deleting seldondeployments seldon.mymab      
INFO removing environment component                component-name=mymab
INFO Removing component parameter references ...  
INFO Deleting component "mymab"                   
INFO Successfully deleted component 'mymab'       


## Tear down

In [38]:
!rm -rf my-ml-deployment